In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
# ANN model with 2 hidden layers

print(f"{'~'*15} Building ANN model with 2 hidden layers {'~'*15}\n")

# Load the dataset
df = pd.read_csv("Datasets/COALINDIA.csv")

# Feature selection and preprocessing
dropped_features = ['Date', 'Symbol', 'Series', 
                    'Trades', 'Turnover', 'Deliverable Volume', 
                    '%Deliverble', 'Last', 'VWAP', 'Prev Close']
df.drop(dropped_features, axis=1, inplace=True)

# Define features (X) and target (Y)
X = df.drop('Close', axis=1)
Y = df['Close']

# Scale the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X.values)
Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

~~~~~~~~~~~~~~~ Building ANN model with 2 hidden layers ~~~~~~~~~~~~~~~



In [18]:
def create_model_A(activation='relu', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, dropout=0.15):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(hidden, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [19]:
ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for dropout in [0.05, 0.1, 0.15]:
                        mse, r2 = create_model_A(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, dropout=dropout)
                        ret.append([activation, epochs, batch_size, op_learning_rate, hidden, dropout, mse, r2])
    

17/17 [==============================] - 0s 409us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error: 142.17835507040726
R^2: 0.9271996
----------------------------------------------------------------
17/17 [==============================] - 0s 385us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.0001
Mean Squared Error: 94.39597152096316
R^2: 0.9516659
----------------------------------------------------------------
17/17 [==============================] - 0s 428us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.001
Mean Squared Error: 3446.4584452611357
R^2: -0.76470995
----------------------------------------------------------------
17/17 [==============================] - 0s 345us/step


Activation: relu
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error: 139.4761120443402
R^2: 0.92858326
----------------------------------------------------------------
17/17 [==============================] - 0s 366us/step


...

Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error: 589.2601170335405
R^2: 0.6982777
----------------------------------------------------------------
17/17 [==============================] - 0s 355us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.0001
Mean Squared Error: 265.7432166542904
R^2: 0.86393
----------------------------------------------------------------
17/17 [==============================] - 0s 364us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Mean Squared Error: 3794.0376580588
R^2: -0.942683
----------------------------------------------------------------
17/17 [==============================] - 0s 354us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 1e-05
Mean Squared Error: 302.2545755452974
R^2: 0.8452349
----------------------------------------------------------------
17/17 [==============================] - 0s 357us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.0001
Mean Squared Error: 259.55662005663316
R^2: 0.86709774
----------------------------------------------------------------
17/17 [==============================] - 0s 412us/step
Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Mean Squared Error: 3973.5761916614415
R^2: -1.0346131
----------------------------------------------------------------


In [22]:
print("Best MSE for ANN:", sorted(ret, key=lambda x:x[7])[0])
print("Best R^2 for ANN:", sorted(ret, key=lambda x:-x[8])[0])

['tanh', 100, 24, 1e-05, 26.320219587223782, 0.9865231]


In [23]:
print(sorted(ret, key=lambda x:-x[7])[0])

['tanh', 100, 24, 1e-05, 26.320219587223782, 0.9865231]


In [26]:
def create_model_P(activation=None, epochs=100, batch_size=32, op_learning_rate=1e-5):
    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation=activation))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    # Calculate MSE
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error (Perceptron): {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2


In [27]:
ret = []
for activation in [None, 'relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                mse, r2 = create_model_P(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate)
                ret.append([activation, epochs, batch_size, op_learning_rate, mse, r2])
    

17/17 [==============================] - 0s 400us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 3659.344585896918
R^2: -0.8737154
----------------------------------------------------------------
17/17 [==============================] - 0s 306us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 446.8241281785368
R^2: 0.7712101
----------------------------------------------------------------
17/17 [==============================] - 0s 433us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 10377.447823250888
R^2: -4.3136253
----------------------------------------------------------------
17/17 [==============================] - 0s 398us/step


Activation: None
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 25647.733277543215
R^2: -12.132557
----------------------------------------------------------------
17/17 [==============================] - 0s 337us/step


...

Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 297.1292265474053
R^2: 0.8478592
----------------------------------------------------------------
17/17 [==============================] - 0s 363us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 2690.6063823107747
R^2: -0.37768686
----------------------------------------------------------------
17/17 [==============================] - 0s 375us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 2894.5116673749085
R^2: -0.48209357
----------------------------------------------------------------
17/17 [==============================] - 0s 329us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 231.00690732064027
R^2: 0.8817162
----------------------------------------------------------------
17/17 [==============================] - 0s 333us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 526.6050978658874
R^2: 0.7303593
----------------------------------------------------------------
17/17 [==============================] - 0s 331us/step
Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 2934.338228334408
R^2: -0.50248635
----------------------------------------------------------------


In [ ]:
print("Best MSE for Perceptron:", sorted(ret, key=lambda x:x[7])[0])
print("Best R^2 for Perceptron:", sorted(ret, key=lambda x:-x[8])[0])

In [28]:
def create_model_D(activation='tanh', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, layers=11, dropout=0.1):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    # Building DNN model with 11 hidden layers
    model = Sequential()

    # Input layer
    # Input layer will have same number of neurons as number of feature variables
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))

    # Hidden layers
    # Play around with number of neurons in each hidden layer.
    # Too many neurons leads to overcomplexity, not enough means too simple
    # Tanh activation function here is used b/c it is recommended to use
    # when there are more hidden layers.
    for _ in range(layers-1):
        model.add(Dense(hidden, activation=activation))
        model.add(Dropout(dropout)) # This helps with preventing overfitting

    # Output layer
    # Output layer will have 1 neuron b/c there's only 1 target variable
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nLayers: {layers}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [29]:
m, r = create_model_D()

17/17 [==============================] - 0s 604us/step
Activation: tanh
Epoch: 100
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 128
Layers: 11
Dropout: 0.1
Mean Squared Error: 27.0655401573587
R^2: 0.9861415
----------------------------------------------------------------


In [ ]:
# GRID SEARCH

ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for layers in [6, 9, 11]:
                        for dropout in [0.05, 0.1, 0.15]:
                            mse, r2 = create_model_D(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, layers=layers, dropout=dropout)
                            ret.append([activation, epochs, batch_size, op_learning_rate, hidden, layers, dropout, mse, r2])
    

In [ ]:
print("Best MSE for DNN:", sorted(ret, key=lambda x:x[7])[0])
print("Best R^2 for DNN:", sorted(ret, key=lambda x:-x[8])[0])